
# 형태소 분석 기반 토큰화의 문제
- 형태소 분석기는 작성된 알고리즘 또는 학습된 내용을 바탕으로 토큰화를 하기 때문에 오탈자나 띄어쓰기 실수, 신조어, 외래어, 고유어 등이 사용된 경우 제대로 토큰화 하지 못한다.
- 그래서 발생 할 수있는 잠재적 문제점
    - 어휘사전을 크게 만든다.
        - 같은 의미의 단어가 형태소 분석이 안되어 여러개 등록될 수있다.
        - ex) 신조어 `돈쭐` 이라는 단어를 인식 못할 경우 `"돈쭐내러", "돈쭐나", "돈쭐냄"` 등이 다 등록 될 수 있다.
    - OOV(Out Of Vocab)에 대응하기 어렵게 만든다.
        - 같은 어근의 단어가 있지만 조사등이 바뀐 신조어등을 OOV로 인식할 수있다.


## 어휘 사전(Vocabulary)과 Out Of Vocabulary (OOV)

- 어휘사전(Vocab)은 토크나이저(Tokenizer)가 사용하는 모든 토큰의 집합이며,**각 토큰**을 고유한 **정수 ID**에 매핑한 사전이다. 토크나이저가 텍스트를 토큰 ID 시퀀스로 변환할 때 기준으로 사용된다. 

   - 매핑된 정수는 모델에 입력되는 텍스트 데이터를 숫자 형식으로 변환해 모델이 처리할 수 있도록 돕는다.
   - 예
        ```bash
        {
            "자연어": 0,
            "처리": 1,
            "는": 2,
            "재미있다": 3,
            "공부": 4,
        }
        ```
- **Out Of Vocabulary (OOV)**
   - 어휘 사전(Vocab): 코퍼스를 구성하는 모든 토큰의 집합.
   - **OOV**란 어휘 사전에 포함되지 않은 토큰을 의미하며, 모델이 해당 토큰을 처리할 수 없기 때문에 일반적으로 특별한 토큰(예: `[UNK]`)으로 대체되거나 다른 방식으로 처리된다.

# Subword Tokenization(하위 단어 토큰화)

## 정의

- Subword Tokenization은 단어를 더 작은 단위(subword)로 나누어 텍스트를 토큰화하는 방식이다.  
    - subword는 하나의 단어를 구성하는 단어들을 말한다.(coworker: co, work, er)
- 주로 자주 등장하는 단어의 일부를 공통된 토큰으로 만들고, 희귀하거나 복합적인 단어는 작은 조각(subword)으로 나누어 처리한다.
- 단어 자체를 그대로 사용하기보다는 단어의 일부를 나누어 처리함으로써 새로운 단어나 미등록 단어(Out-of-Vocabulary) 문제를 줄일 수 있다.

## 장점

1. **미등록 단어 처리 가능**  
   -  새로운 단어(신조어, 속어, 고유어등)가 등장해도 미리 정의된 subword를 조합해서 표현할 수 있어 OOV 문제를 줄일 수 있다.  

2. **어휘 크기 축소**  
   - 같은 subword를 여러 단어에서 공유함으로써, 완전한 단어를 사용하는 경우보다 어휘집의 크기를 작게 유지할 수 있다.


## 종류

1. **Byte-Pair Encoding (BPE)**  
   - 자주 등장하는 문자 쌍을 반복적으로 병합해 서브워드를 생성하는 방식.
   - OpenAI의 GPT 모델에 사용된 토크나이저이다.

2. **Unigram**  
   - 빈도기반 확률모델에 따라 subword 단위를 선택하는 방식이다.  
   - BPE보다 유연하여 더 다양한 분할 결과를 얻을 수 있다.

3. **WordPiece**  
   - BPE와 유사하지만, 빈도수가 아니라, 가능성이 높은 조합(합쳐질 가능성이 높은 subword)에 기반해 subword들을 찾는다.
   - Google의 BERT 모델에 사용된 토크나이저이다.

# Byte Pair Encoding 방식

- 원래 Text data 압축을 위해 만들어진 방법으로 text 에서 자주 같이 나오는 두 글짜 쌍을 합쳐서 하나의 부호(기호,글자)로 만들어 나가면서 글자 수를 줄이는 알고리즘이다. 
- 연속된 글자 쌍이 더 나타나지 않거나 정해진 어휘사전 크기에 도달 할 때 까지 조합을 찾아 부호화 하는 작업을 반복한다.

## text 압축 방식의 예
- 원문: abracadabra
1. AracadAra: ab -> A :=> 원문에서 가장 빈도수 많은 ab를 A(부호로 아무 글자나 사용할 수 있다.)로 치환
2. ABcadAB: ra -> B :=> 1에서 가장 빈도수가 많은 ra를 B로 치환
3. CcadC: AB -> C :=> 2에서 가장 빈도수 맣은 AB를 C로 치환한다.(치환된 글자 쌍도 변환대상에 포함된다.)

## BPE Tokenizer 방식
BPE 토크나이저는 자주 등장하는 글자 쌍을 찾아 치환하는 대신 **단어 사전**에 추가한다.

### 예)
1. 말뭉치의 토큰들의 빈도수, 어휘사전은 아래와 같을 경우
    - 빈도사전: ('low', 5), ('lower', 2), ('newest', 6), ('widest', 3)
    - 어휘사전: ['low', 'lower', 'newest', 'widest']
2. 빈도 사전내의 모든 단어들을 글자 단위로 나눈다. (Pre Tokenization)
    - 빈도사전: ('l', 'o', 'w',  5), ('l', 'o', 'w', 'e', 'r', 2), ('n', 'e', 'w', 'e', 's', 't', 6), ('w', 'i', 'd', 'e', 's', 't', 3)
    - 어휘사전: ['d', 'e', 'i', 'l', 'n', 'o', 'r', 's', 't', 'w']
3. 빈도 사전을 기준으로 가장 자주 등장하는 글자 쌍(byte pair)를 찾는다.  위에서는 **'e'와 's'가 총 9번으로 가장 많이 등장함**. 'e'와 's'를 'es'로 합치고 어휘 사전에 추가한다.
    - 빈도사전: ('l', 'o', 'w',  5), ('l', 'o', 'w', 'e', 'r', 2), ('n', 'e', 'w', **'es'**, 't', 6), ('w', 'i', 'd', **'es'**, 't', 3)
    - 어휘사전: ['d', 'e', 'i', 'l', 'n', 'o', 'r', 's', 't', 'w', **'es'**]
4. 3 번의 과정을 계속 반복한다. 빈도수가 가장 많은 'es'와 't' 쌍을 'est'로 병합하고 'est'를 어휘 사전에 추가한다.
    - 빈도사전: ('l', 'o', 'w',  5), ('l', 'o', 'w', 'e', 'r', 2), ('n', 'e', 'w', **'est'**, 6), ('w', 'i', 'd', **'est'**, 3)
    - 어휘사전: ['d', 'e', 'i', 'l', 'n', 'o', 'r', 's', 't', 'w', **'es'**, **'est'**]
5. 만약 10번 반복했다고 하면 다음과 같은 빈도 사전과 어휘 사전이 생성된다.
    - 빈도 사전: (**'low'**, 5), (**'low'**, 'e', 'r', 2), ('n', 'e', 'w', **'est'**, 6), ('w', 'i', 'd', **'est'**, 3)
    - 어휘사전: ['d', 'e', 'i', 'l', 'n', 'o', 'r', 's', 't', 'w', **'es'**, **'est'**, **'lo'**,**'low'**, **'ne'**, **'new'**, **'newest'**, **'wi'**, **'wid'**, **'widest'**]

- 위와 같이 어휘 사전이 만들어 지면 원래 어휘서전에 없던 것들에 대한 처리를 할 수있다.
    - ex)
        - 'newer' :=> 'new', 'e', 'r', 
        - 'lowest' :=> 'low', 'est'
        - 'wider' :=> 'wid', 'e', 'r'

# WordPiece tokenizer

- Byte Pair Encoding 이 빈도 기반이라면 wordpiece tokenizer는 확률 기반으로 글자 쌍을 병합한다.
- 두개 글자 쌍의 빈도수를 각 개별 글자 빈도수의 곱으로 나눈 점수가 가장 높은 순서대로 글자쌍을 묶어 나간다.

$$
score = \cfrac{f(x, y)}{f(x)\cdot f(y)} 
$$

함수 f는 빈도를 나타내며 x, y는 병합하려는 하위 단어이다.

- 빈도사전: ('l','o','w', 5), ('l','o','w', 'e', 'r', 2), ('n', 'e', 'w', 'e', 's', 't', 6), ('w', 'i', 'd', 'e', 's', 't', 3)
- 어휘사전: ('d', 'e', 'i', 'l', 'n', 'o', 'r', 's', 't', 'w')
- 가장 빈도수가 높은 쌍은 'e','s'로 9번 등장한다. 이때 각 글자는 전체에서 각각 'e'는 17번, 's'는 9번 등장한다. 위 공식에 대입하면 score는 $\frac{9}{17 \times 9} \approx 0.06$ 이다.
- 'i'와 'd' 쌍은 3번만 등장하지만 전체에서 각각 'i' 3번, 'd' 3번 등장한다. 그래서 score는 $\frac{3}{3 \times 3} \approx 0.33$ 이다.
- 나타난 빈도수는 'es' 가 많치만 더 높은 score를 가지는 'id' 쌍을 병합한다.
- 빈도사전: ('l','o','w', 5), ('l','o','w', 'e', 'r', 2), ('n', 'e', 'w', 'e', 's', 't', 6), ('w', **'id'**, 'e', 's', 't', 3)
- 어휘사전: ('d', 'e', 'i', 'l', 'n', 'o', 'r', 's', 't', 'w', **'id'**)
위의 작업을 반복해 연속된 글자 쌍이 더이상 나타나지 않거나 어휘 사전 max 크기에 도달할 때 까지 학습한다.

# Unigram 방식
- 빈도 기반 확률 모델을 사용하여 효율적으로 서브워드를 선택하고, 불필요한 서브워드를 제거해 최적의 어휘 크기를 찾는 알고리즘


- **초기 어휘 집합 구성**
    - 대상 text에 모든 단어와 그 서브스트링을 포함한 어휘 집합을 생성한다. 이 어휘 집합은 나올 수있는 모든 subword들을 다 모아놓은 것이다. 
    - 예를 들어 "hug" 단어의  ["h", "u", "g", "hu", "ug", "hug"]  substring을 만든다. 이들이 subword 후보가 된다.
- **각 Subword의 빈도수 기반 확률 계산**
    -  $\cfrac{subword가\;나타난\;횟수}{전체\;빈도수}$ 로 각 subword들의 나타난 확률을 계산한다.
- **가능한 분할에 대한 확률 계산**
    - 단어를 여러 서브워드로 분할할 수 있는 경우, 각 분할에 대한 전체 확률을 계산한다.
    - 확률 계산은 $ P(subword1)\;\times \; P(subword2)\;\times\; ..$ 으로 계산한다.
    - 예를 들어 "hug" 를 분할 한다고 했을 때
        1. \["h", "u", "g"\]: $ P(h) \times P(u) \times P(g) $
        2. \["hu", "g"\]: $ P(hu) \times P(g) $

   - 각각의 확률을 계산한 후, **가장 높은 확률**을 가진 분할을 선택한다.
     - 위 예에서 만약 1의 확률이 0.01 이고 2의 확률이 0.00001 이라면 첫번째 분할이 선택된다.

- **서브워드 제거**
    - 위의 훈련 과정에서 불필요한 서브워드를 제거하면서 최적의 어휘 집합을 찾아간다. 
    - 제거 대상은 빈도수가 낮거나 조합에 크게 영향을 주지 않은 subword들이다.

> ### korpora 말뭉치
> - 다양한 한글 데이터셋을 제공하는 패키지
> - `pip install korpora`

In [6]:
!uv pip install korpora

Audited 1 package in 281ms


In [7]:
from Korpora import Korpora

corpus = Korpora.load("korean_petitions")
corpus


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : Hyunjoong Kim lovit@github
    Repository : https://github.com/lovit/petitions_archive
    References :

    청와대 국민청원 게시판의 데이터를 월별로 수집한 것입니다.
    청원은 게시판에 글을 올린 뒤, 한달 간 청원이 진행됩니다.
    수집되는 데이터는 청원종료가 된 이후의 데이터이며, 청원 내 댓글은 수집되지 않습니다.
    단 청원의 동의 개수는 수집됩니다.
    자세한 내용은 위의 repository를 참고하세요.

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/

[Korpora] Corpus `korean_petitions` is already installed at C:\Users\USER\Korpora\korean_petitions\petitions_2017-08
[Korpora] Corpus `korean_petitions` is already installed at C:\Users\USER\Korpora\korean_petitions\petitions_2017-09
[Korpora] Corpus `korean_petitions` is already installed at C:

In [8]:
petitions = corpus.get_all_texts()
type(petitions), len(petitions) # 청와대청원 데이터셋. list[str] str: 개별청원

(list, 433631)

In [9]:
print(petitions[0])

안녕하세요. 현재 사대, 교대 등 교원양성학교들의 예비교사들이 임용절벽에 매우 힘들어 하고 있는 줄로 압니다. 정부 부처에서는 영양사의 영양'교사'화, 폭발적인 영양'교사' 채용, 기간제 교사, 영전강, 스강의 무기계약직화가 그들의 임용 절벽과는 전혀 무관한 일이라고 주장하고 있지만 조금만 생각해보면 전혀 설득력 없는 말이라고 생각합니다. 학교 수가 같고, 학생 수가 동일한데 영양교사와 기간제 교사, 영전강 스강이 학교에 늘어나게 되면 당연히 정규 교원의 수는 줄어들게 되지 않겠습니까? 기간제 교사, 영전강, 스강의 무기계약직화, 정규직화 꼭 전면 백지화해주십시오. 백년대계인 국가의 교육에 달린 문제입니다. 단순히 대통령님의 일자리 공약, 81만개 일자리 창출 공약을 지키시고자 돌이킬 수 없는 실수는 하지 않으시길 바랍니다. 세계 어느 나라와 비교해도, 한국 교원의 수준과 질은 최고 수준입니다. 고등교육을 받고 어려운 국가 고시를 통과해야만 대한민국 공립 학교의 교단에 설 수 있고, 이러한 과정이 힘들기는 하지만 교원들이 교육자로서의 사명감과 자부심을 갖고 교육하게 되는 원동력이기도 합니다. 자격도 없는 비정규 인력들을 일자리 늘리기 명목 하에 학교로 들이게 되면, 그들이 무슨 낯으로 대한민국이 '공정한 사회' 라고 아이들에게 가르칠 수 있겠습니까? 그들이 가르치는 것을 학부모와 학생들이 납득할 수 있겠으며, 학생들은 공부를 열심히 해야하는 이유를 찾을 수나 있겠습니까? 열심히 안 해도 떼 쓰면 되는 세상이라고 생각하지 않겠습니까? 영양사의 영양교사화도 재고해주십시오. 영양사분들 정말 너무나 고마운 분들입니다. 학생들의 건강과 영양? 당연히 성장기에 있는 아이들에게 필수적이고 중요한 문제입니다. 하지만 이들이 왜 교사입니까. 유래를 찾아 볼 수 없는 영양사의 '교사'화. 정말 대통령님이 생각하신 아이디어라고 믿기 싫을 정도로 납득하기 어렵습니다. 중등은 실과교과 교사가 존재하지요? 초등 역시 임용 시험에 실과가 포함돼 있으며 학교 현장에서도 정규 교원이 직접 

In [10]:
# 파일로 저장

import os 
os.makedirs('data/petitions', exist_ok=True)
with open("data/petitions/petition_corpus.txt", "wt", encoding="utf-8") as fw:
    for p in petitions:
        fw.write(p+"\n")

In [11]:
# 로드 : 문자열로 load

with open("data/petitions/petition_corpus.txt", encoding = "utf-8") as fr:
    petitions_txt = fr.read()


In [12]:
petitions_txt[:100]

"안녕하세요. 현재 사대, 교대 등 교원양성학교들의 예비교사들이 임용절벽에 매우 힘들어 하고 있는 줄로 압니다. 정부 부처에서는 영양사의 영양'교사'화, 폭발적인 영양'교사' 채용,"

# Hugging Face tokenizers 패키지 사용해 토큰화 수행

## 주요 라이브러리 
- [tokenizers](https://huggingface.co/docs/tokenizers/index)
    - huggingface에서 개발한 tokenizer 라이브러리로 BPE, WordPiece, Unigram 알고리즘을 지원한다. 
    - 설치: `pip install tokenizers`
- [Sentencepiece](https://github.com/google/sentencepiece)
    - 구글에서 개발한 subword tokenizer 라이브러리로 BPE, Unigram 방식 지원.
    - 설치: `pip install sentencepiece`

## Hugging Face tokenizers 패키지이용
- Hugging face는 
  - Subword방식의 토크나이저를 생성할 수 있는 알고리즘을 제공한다. `tokenizers` library를 이용해 사용할 수 있다.
  - GPT, BERT등 다양한 LLM 모델에서 사용된 Pretrained Tokenizer들을 제공한다. `transformers` Library를 이용해 사용할 수 있다.
- 설치: `pip install tokenizers`
- Tokenizer 생성
    - 토큰화 알고리즘을 지정해 instance 생성.
- Trainer 생성
    - 학습 파라미터를 설정해서 instance 생성
- Tokenizer 학습
    - train() 메소드: 학습 text 파일 경로를 지정해서 학습
    - train_from_iterator() 메소드: 학습할 string들을 iterator를 통해 제공.
- https://github.com/huggingface/tokenizers

In [13]:
!uv pip install tokenizers 

Audited 1 package in 580ms


In [14]:
import time

# Tokenizer 어휘사전을 바탕으로 토큰화를 처리하는 객체

from tokenizers import Tokenizer
from tokenizers.models import BPE

#pre tokenixers 로 Subword 방식 토큰화 전에 미리 나누는 방식을 지정.
from tokenizers.pre_tokenizers import Whitespace # 공백기준으로 나눔

#Trainer (토크나이저 모델 학습기) - 알고리즘에 맞춰 trainer를 제공.
from tokenizers.trainers import BpeTrainer

In [15]:
# 토크나이저 객체 생성
tokenizer = Tokenizer(
    BPE(unk_token="[UNK]") # Unknown Token(모르는 단어를 표현할 토큰)을 설정.
)

# 토크나이저에 Pre tokenizer 설정
## BPE 알고리즘으로 토큰화 하기 전에 적용할 나누는 방법 - Whitespace(): 공백 기준으로 미리 나눈다.
tokenizer.pre_tokenizer = Whitespace()

In [ ]:
#Trainer 생성 - 학습 하이퍼파라미터들을 설정

trainer = BpeTrainer(
    vocab_size=10000, # 어휘 사전의 최대 크기
    min_frequency=10, # 최소 출연횟수. 지정한 횟수 (10) 이하로 나온 쌍 (pair)는ㄴ 단어 사전에서 제외.
    special_tokens=["[UNK]","[PAD]" ], # 어휘사전에 추가할 특수목적 토큰. unk_toke은 반드시 추가한다.
   # continuing_subword_prefix="**u"
   # #단으의 시작이 아니라 연결 subword의 앞에 붙이는 prefix
   # BpeTrainer default : 안 붙인다. ("")
   # WordPieceTrainer : '##' 이 디폴트
   # UnigramTrainer: '""' 이 디폴트. 안 붙인다.
)

#Special Token
# 특수목적 토큰
# 토크나이저나 모델에서 사용할 특정 목적의 토큰으로 사용자가 직접 어휘사긴에 추가 해야 한다.
# 대표적인 스페셜 토큰들. 보통 [] 나 <>로 묶어준다.
# oov 표시 토큰 : <unk>, [unk]로 표현
# Padding 토큰 : <pad> 로 표현. 글자 수를 맞춰주기 위해 글자수가 적은 문장에 추가하는 토큰
# 문서의 시적 : <sos> 로 표현
# 문서의 끝 : <eos>
# <cls> : 문서의 시작 + 전체 문서의 의미를 표현 (BERT 모델이 사용하는 특수 토큰)
# 문서 내 일부 토큰을 가리기 : <mask>

In [17]:
# 학습
s = time.time()

tokenizer.train(["data/petitions/petition_corpus.txt"], trainer=trainer)

print("학습에 걸린 시간 :", time.time() - s, "초")

학습에 걸린 시간 : 20.299530029296875 초


In [18]:
# 토크나이저 파일로 저장 > 어휘사전 + 설정들이 저장. (json)
import os

os.makedirs('saved_models', exist_ok=True)
save_path = "saved_models/petitions_bpe.json"
tokenizer.save(save_path)

In [19]:
# 저장된 토크나이저 불러오기
from tokenizers import Tokenizer
load_tokenizer = Tokenizer.from_file(save_path)

In [20]:
# vocab size (토큰(어휘)개수)
tokenizer.get_vocab_size()

10000

In [21]:
# vocab 조회
tokenizer.get_vocab() #딕셔너리로 반환

{'⇚': 550,
 '洲': 2545,
 '淳': 2575,
 '주': 6575,
 'ц': 261,
 '륀': 5083,
 '슶': 5896,
 '见': 3311,
 '몼': 5251,
 '턱': 7152,
 '어도': 8864,
 '따른': 9431,
 '벹': 5435,
 '¹': 116,
 '훓': 7592,
 '쨕': 6693,
 '⊝': 584,
 '瓜': 2766,
 'в': 244,
 '롱': 5036,
 '곫': 4029,
 '庫': 1955,
 '집을': 9427,
 '③': 606,
 '皓': 2827,
 '쟀': 6457,
 '대로': 8275,
 '피해자가': 9730,
 'ㄺ': 1064,
 '낧': 4356,
 '患': 2068,
 '開': 3675,
 '塘': 1684,
 '시작': 8684,
 '좼': 6552,
 '賛': 3450,
 '원에서': 9724,
 '읹': 6401,
 '촛불': 9625,
 '쾈': 7052,
 '군요': 8796,
 '尔': 1853,
 '칠': 6955,
 'ǒ': 166,
 '├': 683,
 '係': 1303,
 '\uf06e': 7705,
 '팡': 7283,
 '톡': 7186,
 '敦': 2253,
 '채': 6811,
 '惹': 2081,
 '렴': 5014,
 '존경': 9737,
 '狂': 2716,
 '뚜': 4885,
 '皇': 2826,
 '덨': 4627,
 '境': 1688,
 '쩨': 6716,
 '쩠': 6713,
 '吏': 7831,
 '행사': 9632,
 '易': 2297,
 '滯': 2612,
 '풰': 7382,
 '쏱': 6009,
 'Ｄ': 7889,
 'q': 82,
 '他': 1249,
 '壇': 1697,
 '曹': 2334,
 '\x81': 96,
 '个': 1184,
 '遵': 3583,
 '곯': 4032,
 'د': 279,
 '̥': 194,
 '마': 5140,
 '微': 2021,
 '훨': 7601,
 '告': 1587,
 '환': 75

In [22]:
# 테스트 문장
sports_txt = "프리미어리그 역대 개인 최다골 기록을 보유하고 있는 시어러가 손흥민의 골 결정력을 재차 극찬했다."
petition_txt = "이 글을 쓴 이유는 다름아닌 '전안법'시행 반대를 주장하기 위해서입니다. 먼저, '전안법'은 전기용품 및 생활용품을 판매하는 업체에서 KC인증마크를 의무적으로 받는 것입니다."
comment_txt = "멋진 식사를 즐기기에 좋은 장소 - 채식 메뉴가 정말 훌륭했습니다. 당근 케이크는 아마도 내가 먹어본 디저트 중 최고였을 거예요."

In [23]:
# 토큰화 : text(str) > token들
token_output = tokenizer.encode(sports_txt)
token_output #Encoding

Encoding(num_tokens=34, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [24]:
# Encoding (토큰화 결과) 에서 각 토큰들 문자열 조회
token_output.tokens

['프',
 '리',
 '미',
 '어',
 '리',
 '그',
 '역',
 '대',
 '개인',
 '최',
 '다',
 '골',
 '기록',
 '을',
 '보유',
 '하고',
 '있는',
 '시',
 '어',
 '러',
 '가',
 '손',
 '흥',
 '민',
 '의',
 '골',
 '결정',
 '력을',
 '재',
 '차',
 '극',
 '찬',
 '했다',
 '.']

In [25]:
# 각 토큰들의 id(정수)로 반환
token_output.ids

[7390,
 5123,
 5330,
 6140,
 5123,
 4128,
 6180,
 4589,
 8414,
 6891,
 4563,
 4027,
 9449,
 6364,
 9442,
 8209,
 8219,
 5908,
 6140,
 4980,
 3902,
 5802,
 7638,
 5332,
 6384,
 4027,
 8940,
 8644,
 6449,
 6793,
 4129,
 6795,
 8565,
 15]

In [26]:
token_output2 = tokenizer.encode(petition_txt)
print(token_output2)
print(token_output2.tokens)

Encoding(num_tokens=52, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
['이', '글을', '쓴', '이유는', '다', '름', '아닌', "'", '전', '안', '법', "'", '시행', '반대', '를', '주장', '하기', '위해서', '입니다', '.', '먼저', ',', "'", '전', '안', '법', "'", '은', '전기', '용', '품', '및', '생활', '용', '품', '을', '판매', '하는', '업체', '에서', 'K', 'C', '인', '증', '마', '크', '를', '의무', '적으로', '받는', '것입니다', '.']


In [27]:
token_output2.ids

[6396,
 8577,
 6044,
 9246,
 4563,
 5107,
 8323,
 8,
 6476,
 6073,
 5412,
 8,
 8656,
 8658,
 5101,
 8953,
 8301,
 8580,
 8212,
 15,
 8855,
 13,
 8,
 6476,
 6073,
 5412,
 8,
 6360,
 9353,
 6278,
 7375,
 5345,
 8437,
 6278,
 7375,
 6364,
 9118,
 8208,
 8692,
 8210,
 44,
 36,
 6400,
 6625,
 5140,
 7091,
 5101,
 8521,
 8253,
 8546,
 8299,
 15]

In [28]:
## 토큰 문자열 > 토큰 ID
tokenizer.token_to_id("이유는")

9246

In [29]:
tokenizer.id_to_token(9246)

'이유는'

In [30]:
#  토큰 id 리스트 > 문장 (str)
decode_output = tokenizer.decode(token_output2.ids)
decode_output

"이 글을 쓴 이유는 다 름 아닌 ' 전 안 법 ' 시행 반대 를 주장 하기 위해서 입니다 . 먼저 , ' 전 안 법 ' 은 전기 용 품 및 생활 용 품 을 판매 하는 업체 에서 K C 인 증 마 크 를 의무 적으로 받는 것입니다 ."

In [31]:
###########################
# Wordpiece 방식으로 학습
#########################
from tokenizers.models import WordPiece #,BPE
from tokenizers.trainers import WordPieceTrainer #BpeTrainer

#토크나이저 생성
wp_tokenizer = Tokenizer(
    WordPiece(unk_token='<unk>')
)
# pre tokenizer 설정
wp_tokenizer.pre_tokenizer = Whitespace()
# Trainer 설정
trainer = WordPieceTrainer(
    vocab_size=20000,
    special_tokens=["<unk>", "<pad>", "<eos>", "<sos>", "<mask>", "<sep>"]
)

In [32]:
s = time.time()
wp_tokenizer.train(["data/petitions/petition_corpus.txt"], trainer)
print(time.time() - s, "초")

43.319226026535034 초


In [33]:
wp_tokenizer.save("saved_models/petitions_wordpice.json")

In [34]:
load_wp_tokenizer = Tokenizer.from_file("saved_models/petitions_wordpice.json")

In [35]:
wp_tokenizer.get_vocab_size() #총 어휘 수

20000

In [36]:
wp_tokenizer.id_to_token(2000) #id > 토큰 문자열 조회

'彗'

In [37]:
wp_tokenizer.token_to_id('彗') # 토큰 문자열 > id 조회

2000

In [38]:
# 인코딩
encoding = wp_tokenizer.encode(sports_txt)
encoding

Encoding(num_tokens=34, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [39]:
encoding.tokens
# ##토큰 : 연결 subword(토큰) 구분자.
# ##이 없는 토큰은 원래 문자열로 되돌릴 때 앞에 공백을 붙인다.
# ##이 있는 토큰은 앞 글자에 붙인다.

['프',
 '##리',
 '##미',
 '##어',
 '##리',
 '##그',
 '역',
 '##대',
 '개인',
 '최',
 '##다',
 '##골',
 '기록',
 '##을',
 '보유',
 '##하고',
 '있는',
 '시',
 '##어',
 '##러',
 '##가',
 '손',
 '##흥',
 '##민',
 '##의',
 '골',
 '결정',
 '##력을',
 '재',
 '##차',
 '극',
 '##찬',
 '##했다',
 '.']

In [40]:
wp_tokenizer.decode(encoding.ids)

'프 ##리 ##미 ##어 ##리 ##그 역 ##대 개인 최 ##다 ##골 기록 ##을 보유 ##하고 있는 시 ##어 ##러 ##가 손 ##흥 ##민 ##의 골 결정 ##력을 재 ##차 극 ##찬 ##했다 .'

In [41]:
decode_str = []
for token in encoding.tokens:
    if token.startswith('##'):
        decode_str.append(token[2:])
    else:
        decode_str.append(' '+token)

#print(decode_str)
''.join(decode_str)

' 프리미어리그 역대 개인 최다골 기록을 보유하고 있는 시어러가 손흥민의 골 결정력을 재차 극찬했다 .'

In [42]:
##############################
# Unigram 모델 학습
from tokenizers.models import Unigram
from tokenizers.trainers import UnigramTrainer

# 토크나이저 객체 생성
uni_tokenizer = Tokenizer(
    Unigram()
)

# pre tokenizer 생성
uni_tokenizer.pre_tokenizer = Whitespace()

# trainer 생성
trainer = UnigramTrainer(
    vocab_size=10000,
    special_tokens=["<unk>","<pad>"],
    min_frequency = 10
)

In [44]:
import time
s = time.time()

uni_tokenizer.train(["data/petitions/petition_corpus.txt"], trainer = trainer)

print(time.time() - s, "초")

255.71477246284485 초


In [45]:
uni_tokenizer.save("saved_models/petitions_unigram.json")

In [47]:
load_uni_tokenizer = Tokenizer.from_file("saved_models/petitions_unigram.json")

In [48]:
encoding = uni_tokenizer.encode(comment_txt)
encoding

Encoding(num_tokens=47, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [49]:
encoding.tokens

['멋',
 '진',
 '식',
 '사',
 '를',
 '즐',
 '기',
 '기에',
 '좋은',
 '장',
 '소',
 '-',
 '채',
 '식',
 '메',
 '뉴',
 '가',
 '정말',
 '훌',
 '륭',
 '했습니다',
 '.',
 '당',
 '근',
 '케',
 '이',
 '크',
 '는',
 '아',
 '마',
 '도',
 '내가',
 '먹',
 '어',
 '본',
 '디',
 '저',
 '트',
 '중',
 '최',
 '고',
 '였',
 '을',
 '거',
 '예',
 '요',
 '.']

In [50]:
encoding.ids

[2663,
 89,
 162,
 36,
 10,
 2231,
 25,
 710,
 482,
 60,
 67,
 114,
 354,
 162,
 789,
 2401,
 8,
 250,
 2675,
 2756,
 282,
 2,
 85,
 471,
 1553,
 3,
 509,
 11,
 63,
 127,
 9,
 1192,
 435,
 41,
 311,
 740,
 123,
 300,
 68,
 739,
 13,
 978,
 4,
 71,
 298,
 69,
 2]